# parameters
> Parameters that are updated by the user
- order: 11

In [ ]:
#| default_exp parameters

In [ ]:
!which python

/Users/kobus/aishiftscheduler/.venv/bin/python


In [ ]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import defaultdict
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from copy import copy
# import time
# import math
# from pprint import pprint
## !pip install -U "ray"
# import ray
# import json

from fastcore.basics import patch
import aishiftscheduler.config as cf
# import aishiftscheduler.policy as pol
# import aishiftscheduler.visualization as vis

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [ ]:
pd.__version__

'2.0.3'

In [ ]:
#| export
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
## SNAMES = [ #state variable names
#     'RAvail_t', #available resource
#     'R_t',      #resource
#     'D_t',      #demand
# ]
## xNAMES = ['x_t'] #decision variable names

In [ ]:
## math parameters use 'math/small case' (as opposed to code parameters)

In [ ]:
## SNAMES = [ #state variable names
#     'RAvail_t', #available resource
#     'R_t',      #resource
#     'D_t',      #demand
# ]
## xNAMES = ['x_t'] #decision variable names

In [ ]:
## math parameters use 'math/small case' (as opposed to code parameters)

In [ ]:
#| export
class Parameters(): pass

In [ ]:
#| export
@patch
def __init__(self:Parameters, W_fn=None, S__M_fn=None, C_fn=None):
    self.START_DATE_TIME = '2023-12-04'
    ## self.START_DATE_TIME = '2023-10-30T22:00' ##works too
    sd = pd.to_datetime(self.START_DATE_TIME)
    assert sd.strftime('%a')=='Mon'

    self.RESOLUTION = 'HOUR' ## 'BLOCK_8_HOUR', 'HOUR', 'QUARTER_HOUR', 
    self.SLOTS_PER_DAY, self.DATE_TIME_DELTA = self.slots_per_day_and_date_time_delta(self.RESOLUTION)

    self.MAX_DAILY_SLOT_RUN = 8
    assert self.MAX_DAILY_SLOT_RUN<=self.SLOTS_PER_DAY

    self.RESOURCE_TYPES = ['Manager', 'AssistMngr', 'RetailAssoc'] ## <<< ========= INPUT ==============
    
    self.RESOURCE_TYPE_COUNTS = [1, 2, 10] ## <<< ========= INPUT ==============
    assert len(self.RESOURCE_TYPES) == len(self.RESOURCE_TYPE_COUNTS)
    assert len(self.RESOURCE_TYPES) <= cf.MAX_RESOURCE_TYPES
    print(f'\n{len(self.RESOURCE_TYPES)=}')
    print(self.RESOURCE_TYPES)

    ## self.TYPES = ['Courtesy']*7 + ['Stocker']*3 + ['Cleaner']*2 + ['Curbsider']*4
    self.TYPES = []
    for i in range(len(self.RESOURCE_TYPES)):
      additional_types = [self.RESOURCE_TYPES[i]]*self.RESOURCE_TYPE_COUNTS[i]
      for item in additional_types:
        self.TYPES.append(item)
    print(f'\n{len(self.TYPES)=}')
    print(self.TYPES)

    self.RESOURCE_IDS = [ ## <<< ========= INPUT ==============
      'Matt', 
      'Mike', 'Tanner', 
      'Jake', 'James', 'Jane', 'John', 'Jim', 'Jenny', 'Jeremy', 'Judy', 'Julie', 'Jeffrey']
    assert len(self.RESOURCE_IDS) == len(self.TYPES)
    assert len(self.RESOURCE_IDS) <= cf.MAX_RESOURCE_IDS

    ## eventually learn, call thBusyRate
    ## demand/busyness ## <<< ========= INPUT ==============
    ## add demands of 0 up to these values as busyness varies from 0 to 100%
    self.DEMANDS_PER_BUSYNESS = [.005, .008, .02] ##set to [0, 0, 0] to disregard
    self.DEMAND_PER_BUSYNESS = {e: self.DEMANDS_PER_BUSYNESS[i] for i,e in enumerate(self.RESOURCE_TYPES)}
    assert len(self.DEMANDS_PER_BUSYNESS) == len(self.RESOURCE_TYPES)

    ## eventually learn, call thVolumeRate
    ## demand/volume ## <<< ========= INPUT ==============
    self.DEMANDS_PER_VOLUME = [.03, .08, .2] ##set to [0, 0, 0] to disregard
    self.DEMAND_PER_VOLUME = {e: self.DEMANDS_PER_VOLUME[i] for i,e in enumerate(self.RESOURCE_TYPES)}
    assert len(self.DEMANDS_PER_VOLUME) == len(self.RESOURCE_TYPES)

    ## eventually learn, call thRevenueRate
    ## demand/revenue ## <<< ========= INPUT ==============
    self.DEMANDS_PER_REVENUE = [.00005, .0001, .0008] ##set to [0, 0, 0] to disregard
    self.DEMAND_PER_REVENUE = {e: self.DEMANDS_PER_REVENUE[i] for i,e in enumerate(self.RESOURCE_TYPES)}
    assert len(self.DEMANDS_PER_REVENUE) == len(self.RESOURCE_TYPES)

    ## what it costs to pay the human resources being scheduled
    self.RESOURCE_EXPENSES = [25.00, 20.00, 18.00] ## <<< ========= INPUT ==============
    self.RESOURCE_EXPENSE = {e: self.RESOURCE_EXPENSES[i] for i,e in enumerate(self.RESOURCE_TYPES)}
    assert len(self.RESOURCE_EXPENSES) == len(self.RESOURCE_TYPES)

    ## *resource* attribute vectors
    self.aNAMES = [tup[0]+'_'+tup[1] for tup in zip(self.TYPES, self.RESOURCE_IDS)]
    print(f'{len(self.aNAMES)=}')
    print(self.aNAMES)

    ## *demand* attribute vectors
    self.bNAMES = self.RESOURCE_TYPES
    print(f'\n{len(self.bNAMES)=}')
    print(self.bNAMES)

    ## *decision* 'attribute' vectors
    self.abNAMES = [] ##to DEMAND b
    for a in self.aNAMES:
      a0,a1 = a.split('_')
      for b in self.bNAMES:
        if(a0==b):
          abn = (a + '___' + b)
          self.abNAMES.append(abn)
    print(f'\n{len(self.abNAMES)=}')
    print(self.abNAMES)

    self.SIM_T = 60
    ## SIM_MU_D = {bNAMES[0]: 4, bNAMES[1]: 2}
    self.SIM_MU_D = {self.bNAMES[0]: 4, self.bNAMES[1]: 2, self.bNAMES[2]: 2}
    print(f'\n{self.SIM_MU_D=}')
    assert len(self.SIM_MU_D.items())==len(self.bNAMES)
    
    ## SIM_EVENT_TIME_D = {bNAMES[0]: None, bNAMES[1]: None, bNAMES[2]: None, bNAMES[3]: None}
    ## SIM_EVENT_TIME_D = {bNAMES[0]: None, bNAMES[1]: None}
    self.SIM_EVENT_TIME_D = {self.bNAMES[0]: None, self.bNAMES[1]: None, self.bNAMES[2]: None}
    print(f'\n{self.SIM_EVENT_TIME_D=}')
    assert len(self.SIM_EVENT_TIME_D.items())==len(self.bNAMES)
    
    ## SIM_MU_DELTA_D = {bNAMES[0]: None, bNAMES[1]: None, bNAMES[2]: None, bNAMES[3]: None}
    self.SIM_MU_DELTA_D = {self.bNAMES[0]: None, self.bNAMES[1]: None, self.bNAMES[2]: None}
    print(f'\n{self.SIM_MU_DELTA_D=}')
    assert len(self.SIM_MU_DELTA_D.items())==len(self.bNAMES)

    self.LABELS = self.setup_plot_labels()

In [ ]:
## ///////////////////////////
# SLOTS_PER_DAY = {
#     'QUARTER_HOUR': 96,
#     'HOUR': 24,
#     'BLOCK_8_HOUR': 3
#     # 'BLOCK_8_HOUR': {0: '6H', 1: '10H', 2: '8H'}
# }
# DATE_TIME_DELTA = {
#     'QUARTER_HOUR': '15min',
#     'HOUR': '1H',
#     'BLOCK_8_HOUR': '8H'
#     # 'BLOCK_8_HOUR': {0: '6H', 1: '10H', 2: '8H'}
# }
## \\\\\\\\\\

In [ ]:
## ///////////////////////////
# SLOTS_PER_DAY = {
#     'QUARTER_HOUR': 96,
#     'HOUR': 24,
#     'BLOCK_8_HOUR': 3
#     # 'BLOCK_8_HOUR': {0: '6H', 1: '10H', 2: '8H'}
# }
# DATE_TIME_DELTA = {
#     'QUARTER_HOUR': '15min',
#     'HOUR': '1H',
#     'BLOCK_8_HOUR': '8H'
#     # 'BLOCK_8_HOUR': {0: '6H', 1: '10H', 2: '8H'}
# }
## \\\\\\\\\\

In [ ]:
#| export
@patch
def slots_per_day_and_date_time_delta(self:Parameters, resolution):
# RESOLUTION = 'HOUR' ## 'BLOCK_8_HOUR', 'HOUR', 'QUARTER_HOUR', 
# if RESOLUTION == 'QUARTER_HOUR':
#   SLOTS_PER_DAY = 96
#   DATE_TIME_DELTA = '15min'
# elif RESOLUTION == 'HOUR':
#   SLOTS_PER_DAY = 24
#   DATE_TIME_DELTA = '1H'
# elif RESOLUTION == 'BLOCK_8_HOUR':
#   SLOTS_PER_DAY = 3
#   DATE_TIME_DELTA = '8H'
#   ## DATE_TIME_DELTA = {0: '6H', 1: '10H', 2: '8H'} ##maybe in future?
# else:
#   print(f'ERROR: Invalid RESOLUTION: {RESOLUTION}')
    if resolution == 'QUARTER_HOUR':
      slots_per_day = 96
      date_time_delta = '15min'
    elif resolution == 'HOUR':
      slots_per_day = 24
      date_time_delta = '1H'
    elif resolution == 'BLOCK_8_HOUR':
      slots_per_day = 3
      date_time_delta = '8H'
      ## DATE_TIME_DELTA = {0: '6H', 1: '10H', 2: '8H'} ##maybe in future?
    else:
      print(f'ERROR: Invalid resolution: {resolution}')
      slots_per_day = 0
      date_time_delta = 'error'    
    return slots_per_day, date_time_delta

In [ ]:
#| export
@patch
def get_availabilities(self:Parameters, dt):
  avails = \
    cf.EXOG_INFO.loc[
      cf.EXOG_INFO['Date']==dt,
      [col for col in cf.EXOG_INFO.columns if col in [f'A_{a}' for a in range(len(self.RESOURCE_IDS))]]
    ].iloc[0]
  avails.reset_index(drop=True, inplace=True) ##to start index at 0
  return avails

In [ ]:
#| export
@patch
def get_dow_qod_capacities(self:Parameters, dow):
  capacities = \
    cf.EXOG_INFO.loc[
      (cf.EXOG_INFO['DOW']==dow),
      [col for col in cf.EXOG_INFO.columns if col in [f'A_{a}' for a in range(len(self.RESOURCE_IDS))]]
    ].sum(axis=0) ##total daily capacity
  capacities.reset_index(drop=True, inplace=True)
  return capacities

In [ ]:
#| export
@patch
def get_dow_hod_capacities(self:Parameters, dow):
  capacities = \
    cf.EXOG_INFO.loc[
      (cf.EXOG_INFO['DOW']==dow),
    ].groupby(['HOD']).first()
  capacities = capacities[[col for col in cf.EXOG_INFO.columns if col in [f'A_{a}' for a in range(len(self.RESOURCE_IDS))]]]
  capacities = capacities.sum(axis=0) ##total daily capacity
  capacities.reset_index(drop=True, inplace=True)
  return capacities

In [ ]:
#| export
@patch
def get_dow_bod_capacities(self:Parameters, dow):
  capacities = \
    cf.EXOG_INFO.loc[
      (cf.EXOG_INFO['DOW']==dow),
    ].groupby(['BOD']).first()
  capacities = capacities[[col for col in cf.EXOG_INFO.columns if col in [f'A_{a}' for a in range(len(self.RESOURCE_IDS))]]]
  capacities = capacities.sum(axis=0) ##total daily capacity
  capacities.reset_index(drop=True, inplace=True)
  return capacities

In [ ]:
#| export
@patch
def get_capacities(self:Parameters, dow):
  if self.RESOLUTION == 'QUARTER_HOUR':
    return self.get_dow_qod_capacities(dow)
  elif self.RESOLUTION == 'HOUR':
    return self.get_dow_hod_capacities(dow)
  elif self.RESOLUTION == 'BLOCK_8_HOUR':
    return self.get_dow_bod_capacities(dow)
  else:
    print(f'ERROR: Invalid RESOLUTION: {self.RESOLUTION}')
    return None

In [ ]:
#| export
@patch
def setup_plot_labels(self:Parameters):
    RAvail_t_labels = ['RAvail_t_'+an for an in self.aNAMES]
    RCumSlots_t_labels = ['RCumSlots_t_'+an for an in self.aNAMES]
    RCumMerits_t_labels = ['RCumMerits_t_'+an for an in self.aNAMES]
    ## RContSlots_t_labels = ['RContSlots_t_'+an for an in aNAMES]
    RComplete_t_labels = ['RComplete_t_'+an for an in self.aNAMES]
    DSlot_t_labels = ['DSlot_t_'+rt for rt in self.RESOURCE_TYPES]
    DBusy_t_labels = ['DBusy_t_'+rt for rt in self.RESOURCE_TYPES]
    DVolume_t_labels = ['DVolume_t_'+rt for rt in self.RESOURCE_TYPES]
    DRevenue_t_labels = ['DRevenue_t_'+rt for rt in self.RESOURCE_TYPES]
    DMax_t_labels = ['DMax_t_'+rt for rt in self.RESOURCE_TYPES]
    B_t_label = ['B_t']
    V_t_label = ['V_t']
    N_t_label = ['N_t']
    xAlloc_t_labels = ['Allocd_t_'+abn for abn in self.abNAMES]
    labels = ['piName', 'theta', 'l'] + \
      ['t', 'dt'] + \
      RAvail_t_labels + RCumSlots_t_labels + RCumMerits_t_labels + RComplete_t_labels + \
      DSlot_t_labels + \
      DBusy_t_labels + \
      DVolume_t_labels + \
      DRevenue_t_labels + \
      DMax_t_labels + \
      B_t_label + \
      V_t_label + \
      N_t_label + \
      ['Ucum_'+rt for rt in self.RESOURCE_TYPES] + \
      ['Ucum_Total'] + \
      ['Ccum_CumSlots'] + \
      ['Ccum_SickProb'] + \
      ['Ccum_CumMerits'] + \
      ['Ccum_ContSlots'] + \
      ['Ccum'] + \
      xAlloc_t_labels
    return labels

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()